# Database 설정
MySQL 데이터베이스에서 다음 SQL문을 실행하여 데이터베이스를 설정합니다.

```mysql
CREATE DATABASE `project` DEFAULT CHARACTER SET utf8 COLLATE utf8_general_ci;

USE project;
CREATE TABLE `adm_dongs` (
    `code` VARCHAR(7) NOT NULL,
    `address` VARCHAR(256) NOT NULL,
    PRIMARY KEY (`code`)
) ENGINE=InnoDB;

CREATE TABLE `adm_dong_borders` (
    `code` VARCHAR(7) NOT NULL,
    `vertex_index` INT NOT NULL,
    `latitude` FLOAT NOT NULL,
    `longtitude` FLOAT NOT NULL,
    PRIMARY KEY (`code`, `vertex_index`),
    FOREIGN KEY (`code`) REFERENCES `adm_dongs` (`code`)
) ENGINE=InnoDB;
```

In [1]:
import pymysql
import json

In [2]:
f = open('borders.geojson')
raw = f.read()
f.close()
data = json.loads(raw)

In [3]:
conn = pymysql.connect(host='localhost', user='root', password='0712', db='project', charset='utf8')

cursor = conn.cursor()

for feature in data['features']:
    properties = feature['properties']
    code = properties['adm_cd']
    cursor.execute("INSERT INTO `adm_dongs` (`code`, `code2`, `address`) VALUES (%s, %s, %s)", (code, properties['adm_cd2'], properties['adm_nm']))
    
    i = 0
    for vertex in feature['geometry']['coordinates'][0][0]:
        cursor.execute("INSERT INTO `adm_dong_borders` (`code`, `vertex_index`, `latitude`, `longtitude`) VALUES (%s, %s, %s, %s)", (code, i, vertex[1], vertex[0]))
        i += 1

conn.commit()
conn.close()